In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

balaji = pd.read_csv('Balaji Fast Food Sales.csv')
balaji.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   order_id            1000 non-null   int64 
 1   date                1000 non-null   object
 2   item_name           1000 non-null   object
 3   item_type           1000 non-null   object
 4   item_price          1000 non-null   int64 
 5   quantity            1000 non-null   int64 
 6   transaction_amount  1000 non-null   int64 
 7   transaction_type    893 non-null    object
 8   received_by         1000 non-null   object
 9   time_of_sale        1000 non-null   object
dtypes: int64(4), object(6)
memory usage: 78.3+ KB


In [5]:
balaji["transaction_type"] = (
    balaji["transaction_type"]
    .replace(r"^\s*$", pd.NA, regex=True)
    .fillna("Unknown")
)
balaji

,order_id,date,item_name,item_type,item_price,quantity,transaction_amount,transaction_type,received_by,time_of_sale
0,1,07-03-2022,Aalopuri,Fastfood,20,13,260,Unknown,Mr.,Night
1,2,8/23/2022,Vadapav,Fastfood,20,15,300,Cash,Mr.,Afternoon
2,3,11/20/2022,Vadapav,Fastfood,20,1,20,Cash,Mr.,Afternoon
3,4,02-03-2023,Sugarcane juice,Beverages,25,6,150,Online,Mr.,Night
4,5,10-02-2022,Sugarcane juice,Beverages,25,8,200,Online,Mr.,Evening
...,...,...,...,...,...,...,...,...,...,...
995,996,3/19/2023,Frankie,Fastfood,50,10,500,Unknown,Mrs.,Evening
996,997,9/20/2022,Sandwich,Fastfood,60,7,420,Unknown,Mr.,Evening
997,998,1/26/2023,Sandwich,Fastfood,60,13,780,Unknown,Mr.,Evening
998,999,8/27/2022,Panipuri,Fastfood,20,5,100,Unknown,Mrs.,Afternoon
